# **Building a Custom ETL Tool with Python**

ETL tool for integration can either be custom-coded or licensed. One of the tools for building a custom ETL tool is Python. For this exercise we will be building an ETL by integrating data from multiple sources into a single data warehouse as a single source of truth. This exercise will involve extraction from range of sources including:

* Databases
* CSV files
* JSON files
* Excel files

The first step is the extraction, where data is extracted from these sources. These extracted data are then transformed to match the target.

**NOTE**: Before we begin check if you have `psycopg2` installed by running the code below. If you get an error run `!pip install --upgrade psycopg2` in a new cell to fix it.

In [ ]:
import psycopg2 as ps

~~**NOTE 2:** I have put the Eurocontrol Flight Data into an SQL database in the cloud using Google Cloud Platform. For security, there is an "allowed list" of computers that can connect to this database. I need to add your IP address to this "allowed list" before you can connect. Run the code below to find out your IP address.~~

~~`import requests
url = 'https://api.ipify.org'
response = requests.get(url)
response.text`~~

*Update: I have moved this database to a server that does not require extra security and changed the code below to use it*

# Section 1: **Extraction**

In [ ]:
import pandas as pd
import numpy as np
import psycopg2 as ps

### **Extraction of flights table from database**

For us to use it we need to create a connection to the database and extract the information. There are several options for connecting to the database including `SQLAlchemy`, `SQLite3`, and `psycopg2` (a postgres adapter for python). Because we know our database is managed by Postgres, we can connect using `psycopg2`.

In [ ]:
# Create a connection to the database
# UPDATED with new host that does not require extra security

connection = ps.connect(host="35.246.96.119",
                        port="5432",
                        user="readonly",
                        password="bunny2021",
                        dbname="eurocontrol"
                        )

In [ ]:
# Create a so-called "cursor" for selection in the database
cursor = connection.cursor()

We can write SQL commands in Python using the `psycopg2` execute command. Try the cell below to obtain the table names

In [ ]:
query_name='''SELECT table_name
              FROM information_schema.tables
              WHERE table_schema='public'
              ORDER BY table_name'''

In [ ]:
# Fetch the names of tables in the database.
try:
    cursor.execute(query_name)
except:
    connection.rollback()
    cursor.execute(query_name)
for table_name in cursor.fetchall():
    name = table_name[0]
    print(name)

The query below will help us explore the `flights` table. There are nearly 14 million rows in this table; if you try and select them all it will work, however it will take some time to complete as this will download 1.8GB of data on to your computer which is a managable amount but not ideal. So we add `LIMIT 100` to the end of our query to get a flavour of the data to work with. 

In [ ]:
query="""SELECT *
        FROM flights
        LIMIT 100"""

In [ ]:
# before running a new query, we need to "rollback" the previous query
connection.rollback()
# Read the sql query using pandas and database connection
# pandas handles the "rollback" so we do not have to :)
df = pd.read_sql_query(query, connection)
# this has created a dataframe which we can use in the usual way
df

Only having 100 rows is not very useful. Let's instead extract all flights that departed from Manchester airport, this will give us a data set which is small enough to be managable but big enough to be interesting.

In [ ]:
query_manchester = """
SELECT *
FROM flights
WHERE adep = 'EGCC'
"""

In [ ]:
flights = pd.read_sql(query_manchester, connection)

### Explore the flights data using pandas

In [ ]:
flights.head()

In [ ]:
flights.shape

In [ ]:
flights.describe()

In [ ]:
flights.dtypes

We are now going to extract some more information from a variety of sources into this workbook

### **Extraction of airports table from a CSV file**


We have used `pd.read_csv` to get data from a CSV file. Note that you can also use it to download a CSV straight off the internet like in the code below. This downloads data provided by openflights.org see https://openflights.org/data.html for the data dictionary.

Note that even though this is a CSV file, openflights call it `airports.dat` rather than `airports.csv` and I don't know why.

In [ ]:
# header=None because the source data does not have a header row
airports = pd.read_csv('https://raw.githubusercontent.com/jpatokal/openflights/master/data/airports.dat', header=None)

**Explore the airports data**

In [ ]:
# Use .head() .shape .describe() and/or .dtypes in the same way we did above on the flights dataframe


### **Extraction of aircraft table from a JSON file**

Use `pd.read_json`

In [ ]:
aircraft = pd.read_json('aircraft.json')

**Explore the aircraft data**

### **Extraction of weather data from an Excel file** 

Use `pd.read_excel`

If this gives an error about a "Missing optional dependency" then run `!pip install openpyxl` in a new cell to fix this.

In [ ]:
# this can take some time because the file is quite large
weather = pd.read_excel('weather.xlsx')

**Explore the weather data**

See https://dev.meteostat.net/formats.html to understand what each column means. Note that I added the ICAO column myself so this is not listed.

File includes data downloaded for Heathrow, Manchester, Stansted and Gatwick.

### Extension

Experiment extracting information using different SQL queries with psycopg2, e.g. use filters and groupbys

# Section 2: **Transformation**

Now we have extracted the information we need to transform it before we can load it into one file. We will have to explore our data to find out what needs to be transformed.

What we are aiming for is a single file that contains extended information about each flight from Manchester.

## **Transformation of `flights` table**

1. Validate data types are appropriate
1. Check for null values and treat accordingly
1. Check for any other data quality issues i.e. ZZZZ

First check our data types

In [ ]:
# We can use .info() to examine our data, are there any fields which show data types we do not expect?

flights.info()

It is a minor issue but there is no way that requested_fl should have any decimal values, so change this column type to `int64` 

### Exercise

* Check if any of the values are actually floats, and if not, go ahead and change the column type. This is quite difficult so I have included code here for you that checks if any of the values are floats

In [ ]:
rfl_is_float = flights['requested_fl'].map(float.is_integer) == False
flights[rfl_is_float]

In [ ]:
# now change the data type of column requested_fl from float64 to int64


### Exercise

* Check for null values and deal with them accordingly using .isnull().sum()

## **Transformation of `airports` table**

1. Rename columns if needed
1. Drop any obviously redudant columns
1. Validate data type and convert where necessary
1. Check for null values and treat accordingly.

In [ ]:
airports

### Exercise

* rename the columns using the `headings` list below
* drop the `Type` and `Source` columns
* check the data types
* remove the null values (which are represented by `\N` in this data)

In [ ]:
# these are the column headings for this data (source https://openflights.org/data.html)
headings = ['Airport ID','Name','City','Country','IATA','ICAO','Latitude','Longitude','Altitude','Timezone','DST','Tz database time zone','Type','Source']

# use the list above to rename the columns in airports


In [ ]:
# drop the Type and Source columns as these are not useful


In [ ]:
# check the data types


In [ ]:
# remove the null values which are represented as \N by first replacing '\\N' with np.nan then drop the nans
# (double backslash above is *not* a typo because \ is a special character in Python, hence need '\\N')


## Transformation of `aircraft` table

### Exercise

1. Check for and deal with null values
1. Check for any duplicates in the `registration` column and remove these
1. Validate data types and convert if necessary

In [ ]:
# check for nulls


In [ ]:
# check for duplicates in the registration column and remove these


In [ ]:
# validate data types


### Exercise

* Are there any other data quality issues i.e. ZZZ
* What should we do about these?


## Transformation of `weather` table

1. Check for and deal with null values
1. Create a column called `key` which concatenates together the `icao` code + `date` + `hour`. For example the `key` value for the first row would be:

`EGSS 2015-01-01 01`

In [ ]:
# deal with nulls


In [ ]:
# create a new column called key as described above


# Section 3: Loading

Now that the transformation stage is complete, the next step is to load the the data into the data warehouse (or integrate it into one single file). 

Loading could involve `joining`, `concatenating` or just loading directly into a database. For our integration exercise, we will join the data into one single integrated file using pandas join functions. 

We need to identify joining fields on the extracted data.

### Exercise

`flights` is the central table we will join the others to

* Which field in the `aircraft`, `airports` and `weather` tables are foreign keys in `flights`?
* Using join functions, create one large dataframe which contains all the information from the 4 tables
* Explore the large dataframe
* Use `.to_csv()` to save the large dataframe as a CSV file


In [ ]:
# join flights to aircraft
data = flights.join(aircraft.set_index('registration'), on='ac_registration', how='left' )

In [ ]:
data.head()

In [ ]:
# join other tables here


In [ ]:
# export as a CSV file using data.to_csv()


# Loading data into a BigQuery Database (Optional)

We can load our data into the database of our choice. For example we can load it into a BigQuery database. To run the code below, you will need to sign up for google bigquery and create projectgenerate bigquery credentials. There is a free version that you can use. Once you have created a project, you will need to generate the credentials to use for the code below. Details on how to create a google cloud project is [here](https://cloud.google.com/resource-manager/docs/creating-managing-projects) and how to generate credentials [here](https://cloud.google.com/docs/authentication/getting-started)

You also need to install the google cloud BigQuery Library


In [ ]:
!pip install google-cloud-bigquery

In [ ]:
from google.cloud import bigquery

***Do not run the cells below until you have your credentials generated***

In [ ]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = #path to your bigquery credential

In [ ]:
from google.cloud import bigquery

# Construct a BigQuery client object.
client = bigquery.Client()

# Set dataset_id to the ID of the dataset to create.
dataset_id = "{}.warehouse_data".format(client.project)

# Construct a full Dataset object to send to the API.
dataset = bigquery.Dataset(dataset_id)

# Specify the geographic location where the dataset should reside.
dataset.location = "UK"

# Send the dataset to the API for creation.
# Raises google.api_core.exceptions.Conflict if the Dataset already
# exists within the project.
dataset = client.create_dataset(dataset)  # Make an API request.
print("Created dataset {}.{}".format(client.project, dataset.dataset_id))

In [ ]:
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
#
# THESE COLUMN DEFINITIONS ARE FOR THE IOWA LIQUOR DATABASE AND WILL NEED TO BE
# CHANGED TO REFLECT YOUR DATA FILE YOU CREATED
#
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

# Set table_id to the ID of the table to create.
table_id = f"{client.project}.warehouse_data.master_table"

schema = [
    bigquery.SchemaField("date", "DATE", mode="NULLABLE"),
    bigquery.SchemaField("convenience_store", "BOOLEAN", mode="NULLABLE"),
    bigquery.SchemaField("store", "INTEGER", mode="NULLABLE"),
    bigquery.SchemaField("county_number", "INTEGER", mode="NULLABLE"),
    bigquery.SchemaField("county", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("category", "INTEGER", mode="NULLABLE"),
    bigquery.SchemaField("category_name", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("vendor_no", "INTEGER", mode="NULLABLE"),
    bigquery.SchemaField("vendor", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("item", "INTEGER", mode="NULLABLE"),
    bigquery.SchemaField("description", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("pack", "INTEGER", mode="NULLABLE"),
    bigquery.SchemaField("liter_size", "INTEGER", mode="NULLABLE"),
    bigquery.SchemaField("state_btl_cost", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("btl_price", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("bottle_qty", "INTEGER", mode="NULLABLE"),
    bigquery.SchemaField("total", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("item_description", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("vendor_name", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("bottle_size", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("inner_pack", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("proof", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("list_date", "TIMESTAMP", mode="NULLABLE"),
    bigquery.SchemaField("upc", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("scc", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("bottle_price", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("shelf_price", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("case_cost", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("name", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("store_status", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("store_address", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("population", "INTEGER", mode="NULLABLE"),
]

table = bigquery.Table(table_id, schema=schema)
table = client.create_table(table)  # Make an API request.
print(
    "Created table {}.{}.{}".format(table.project, table.dataset_id, table.table_id)
)

In [ ]:
client = bigquery.Client()
filename = "master_data.csv"
dataset_id = table.dataset_id
table_id = table.table_id


dataset_ref = client.dataset(dataset_id)
table_ref = dataset_ref.table(table_id)
job_config = bigquery.LoadJobConfig()
job_config.source_format = bigquery.SourceFormat.CSV
job_config.skip_leading_rows = 1
job_config.autodetect = True

with open(filename, "rb") as source_file:
    job = client.load_table_from_file(source_file, table_ref, job_config=job_config)

job.result()  # Waits for table load to complete.

print("Loaded {} rows into {}:{}.".format(job.output_rows, dataset_id, table_id))

**Now you can log on to your BigQuery to view your data**


**You can also connect to your BigQuery Datawarehouse right from Power BI and create a dashboard to help make business decisions**